<a href="https://colab.research.google.com/github/monta0315/pytorch_pra/blob/main/Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automatic Differential with torch.autograd
ニューラルネットワークを訓練する際、その学習アルゴリズムとしてバックプロパゲーションが使用される

バックプロパゲーションではモデルの重みなどのパラメータは損失関数に対するその変数の微分値に応じて調整されます

これらの勾配の値を計算するためにPytorchにはtorch.autogradという微分エンジンが組み込まれている

autogradはPytorchの計算グラフに対する勾配の自動計算を支援する

シンプルな１レイヤーのネットワークを想定する

入力をx、パラメータをw,b、そして適切な損失関数を決める

In [1]:
%matplotlib inline

In [4]:
import torch

x = torch.ones(5) #inputTensor
y = torch.zeros(3)
w = torch.randn(5,3,requires_grad=True)
b = torch.randn(3,requires_grad=True)
z = torch.matmul(x,w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

上記のコードでは以下の計算グラフを示している
<img src="https://pytorch.org/tutorials/_images/comp-graph.png" width=50% alt="海の写真" title="空と海">


上記のネットワークではw,bが最適化したいパラメータ

そのため、これらの変数に対する損失関数の微分値を計算する必要性がある

これらのパラメータで微分を可能にするためにreaquires_grad属性を追記する

また、requires_gradはテンソルを定義する際、もしくはその後にx.requires_grad_(True)など指定もできる

計算グラフを構築する際にテンソルに適用する関数は実際にはFucntionクラスのオブジェクトである

これらのオブジェクトでは順伝播時に入力をどのように処理するか定義されている

加えてバックプロパゲーション時に勾配をどのように計算するかも把握している

そして、勾配はテンソルのgrad_fnプロパティに格納される

実際にテンソルに適用する関数がバックプロパゲーション時にどのように計算するかを知っているからややこしい記述はいらないってこと？？

In [6]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7fc286e21fd0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7fc286e21d10>


勾配の計算

ニューラルネットワークの各パラメータを最適化するために、入力xと出力yが与えられたもとで、損失関数の各変数の偏微分値を求める必要がある

$\frac{\partial loss}{\partial w}$ 、$\frac{\partial loss}{\partial b}$ 

これらの偏微分値を求めるために、loss.backward()を実行し、w.gradとb.gradの値を導出する必要がある

In [7]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0936, 0.3226, 0.0351],
        [0.0936, 0.3226, 0.0351],
        [0.0936, 0.3226, 0.0351],
        [0.0936, 0.3226, 0.0351],
        [0.0936, 0.3226, 0.0351]])
tensor([0.0936, 0.3226, 0.0351])


勾配計算をしない方法

デフォルトでは全てのテンソルはrequires_grad = Trueであり、計算履歴が保持され、勾配計算可能な状態である

しかし、勾配計算が不要なケースも存在する。

例えば訓練済モデルで推論するケースなど

この場合はネットワークの順伝播関数のみ使用する

実装コードで勾配計算を不要にするためにはtorch.no_gradのブロックを記述する

In [8]:
z = torch.matmul(x,w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x,w)+b
print(z.requires_grad)

True
False


In [9]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


勾配を計算、追跡を不能にしたいケースは下記のような場合

ネットワークの一部のパラメータを固定にしたい(frozen parameters)ケース　これはファインチューニング時にある

順伝搬の計算スピードを高速化したい時